Context
This dataset is originally from the National Institute of Diabetes and
Digestive and Kidney Diseases. The objective is to predict based on
diagnostic measurements whether a patient has diabetes.
Content
Several constraints were placed on the selection of these instances from a
larger database. In particular, all patients here are females at least 21 years
old of Pima Indian heritage.
• Pregnancies: Number of times pregnant
• Glucose: Plasma glucose concentration 2 hours in an oral glucose
tolerance test
• BloodPressure: Diastolic blood pressure (mm Hg)
• SkinThickness: Triceps skin fold thickness (mm)
• Insulin: 2-Hour serum insulin (mu U/ml)
• BMI: Body mass index (weight in kg/(height in m)^2)
• DiabetesPedigreeFunction: Diabetes pedigree function
• Age: Age (years)
• Outcome: Class variable (0 or 1)

1. Prepare the provided Diabetes dataset, implement ONE of the classification techniques and evaluate the performance using the proper metrics for the binary classification of the variable “Outcome”

In [9]:
install.packages("randomForest")



The downloaded binary packages are in
	/var/folders/rd/jfxc79756t9g0g5f4ytyb6l40000gn/T//Rtmpq88GOO/downloaded_packages


In [10]:
library(tidyverse)
library(caret)
library(e1071)
library(randomForest)
library(pROC)

In [14]:
data <- read.csv('diabetes.csv')

In [15]:
head(data)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<int>,<int>
1,6,148,72,35,0,33.6,0.627,50,1
2,1,85,66,29,0,26.6,0.351,31,0
3,8,183,64,0,0,23.3,0.672,32,1
4,1,89,66,23,94,28.1,0.167,21,0
5,0,137,40,35,168,43.1,2.288,33,1
6,5,116,74,0,0,25.6,0.201,30,0


In [18]:
summary(data)

  Pregnancies        Glucose      BloodPressure    SkinThickness  
 Min.   : 0.000   Min.   :  0.0   Min.   :  0.00   Min.   : 0.00  
 1st Qu.: 1.000   1st Qu.: 99.0   1st Qu.: 62.00   1st Qu.: 0.00  
 Median : 3.000   Median :117.0   Median : 72.00   Median :23.00  
 Mean   : 3.845   Mean   :120.9   Mean   : 69.11   Mean   :20.54  
 3rd Qu.: 6.000   3rd Qu.:140.2   3rd Qu.: 80.00   3rd Qu.:32.00  
 Max.   :17.000   Max.   :199.0   Max.   :122.00   Max.   :99.00  
    Insulin           BMI        DiabetesPedigreeFunction      Age       
 Min.   :  0.0   Min.   : 0.00   Min.   :0.0780           Min.   :21.00  
 1st Qu.:  0.0   1st Qu.:27.30   1st Qu.:0.2437           1st Qu.:24.00  
 Median : 30.5   Median :32.00   Median :0.3725           Median :29.00  
 Mean   : 79.8   Mean   :31.99   Mean   :0.4719           Mean   :33.24  
 3rd Qu.:127.2   3rd Qu.:36.60   3rd Qu.:0.6262           3rd Qu.:41.00  
 Max.   :846.0   Max.   :67.10   Max.   :2.4200           Max.   :81.00  
    Outcome  

Glucose
Normal: < 140
Pre-Diabetes: 140-199
Diabetes: > 200

Insulin Ranges
Low: < 25 μU/ml
Normal: 25 - 230 μU/ml
High: > 230 μU/ml

Blood pressure
Normal: < 80
Elevated: 80-89
High: > 90

BMI
Normal: 18.5-24.9
Overweight: 25-29.9
Obesity: >30

Outcome:
0 = no diabetes
1 = diabetes

This is for me to understand some of the data im dealing with since im not in health field. Also printed out summary of data to get a picture of the data that im working with

In [22]:
sum(is.na(data))

[1] 0

Since I have 0s in some of the columns, It means that they have not been recorded so I want to print out the columns with 0s and how many zeros are in there so i can see what i can do with my missing data. Prior to that I am going to count my rows to see how much rows I have in my diabetes file.  

In [25]:
number_of_rows <- nrow(data)
cat("Number of rows in the dataset:", number_of_rows, "\n")

Number of rows in the dataset: 768 


In [27]:
num_zeros_glucose <- sum(data$Glucose == 0)
num_zeros_bp <- sum(data$BloodPressure == 0)
num_zeros_skinthickness <- sum(data$SkinThickness == 0)
num_zeros_insulin <- sum(data$Insulin == 0)
num_zeros_bmi <- sum(data$BMI == 0)

In [29]:
cat("Number of zeros in Glucose:", num_zeros_glucose, "\n")
cat("Number of zeros in BloodPressure:", num_zeros_bp, "\n")
cat("Number of zeros in SkinThickness:", num_zeros_skinthickness, "\n")
cat("Number of zeros in Insulin:", num_zeros_insulin, "\n")
cat("Number of zeros in BMI:", num_zeros_bmi, "\n")

Number of zeros in Glucose: 5 
Number of zeros in BloodPressure: 35 
Number of zeros in SkinThickness: 227 
Number of zeros in Insulin: 374 
Number of zeros in BMI: 11 


High number of missing data from Skin_thickness and Insulin. However, I do not think median would be a good idea for insulin since it can lead to a big wide range of bias. Skin Thickness can somewhat correlate with BMI but I do not worry about skinthickness compared to insulin levels when it comes to diabetis since other factors can play bigger.

In [34]:
outcome_counts <- table(data$Outcome)


cat("Number of non-diabetic (0):", outcome_counts[1], "\n")
cat("Number of diabetic (1):", outcome_counts[2], "\n")

Number of non-diabetic (0): 500 
Number of diabetic (1): 268 


I am checking to get a count of non-diabetic, and diabetic count for my data. I also will be checking how many rows are effected with missing values for skinthickness and insulin in my non-diabetic and diabetic. 

In [36]:

zero_skinthickness_outcome <- table(data$Outcome, data$SkinThickness == 0)

zero_insulin_outcome <- table(data$Outcome, data$Insulin == 0)


cat("Number of non-diabetic (0) with zero SkinThickness:", zero_skinthickness_outcome[1,2], "\n")
cat("Number of diabetic (1) with zero SkinThickness:", zero_skinthickness_outcome[2,2], "\n")
cat("Number of non-diabetic (0) with zero Insulin:", zero_insulin_outcome[1,2], "\n")
cat("Number of diabetic (1) with zero Insulin:", zero_insulin_outcome[2,2], "\n")

Number of non-diabetic (0) with zero SkinThickness: 139 
Number of diabetic (1) with zero SkinThickness: 88 
Number of non-diabetic (0) with zero Insulin: 236 
Number of diabetic (1) with zero Insulin: 138 


I wanted to see what the total count of outcome is for the missing values. 